# Detecting Anomalies using the Wallaroo mobilenet Pipeline in a Video Stream

In this tutorial we will explore using the mobilenet pipeline we created in step 3 to run inference on the frames in a Video and then draw the identified object's bounding boxes, classification and classification confidence.

In [ ]:
import torch
import pickle
import wallaroo
import os
import numpy as np
import json
import requests
import time
import imutils
from CVDemoUtils import CVDemo



In [ ]:
wl = wallaroo.Client()

In [ ]:
ws = wl.list_workspaces()
for w in ws:
    if w.name() == 'computer-vision':
        wl.set_current_workspace(w)

In [ ]:
model_name = 'mobilenet'
mobilenet_model = wl.upload_model('mobilenet', "models/mobilenet.pt.onnx")

Next we will add our post processing anomoly detection file called post-process-anomoly-detection.py.  Predictions that are lower than 75% we will consider anomalies that need to be inspected.

In [ ]:
module_anomoly_detection = wl.upload_model("post-process-anomoly-detection", "./post-process-anomoly-detection.py").configure('python')

### Deploy our custom anomoly detection

Next we will deploy our pipeline with the custom anomoly detection logic as a post processing step in our pipeline.|

In [ ]:
deployment_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(1).memory("8Gi").build()

In [ ]:
pipeline_name = 'anomoly-pp'
pipeline = wl.build_pipeline(pipeline_name) \
            .add_model_step(mobilenet_model) \
            .add_model_step(module_anomoly_detection)

pipeline.deploy(deployment_config = deployment_config)

In [ ]:
time.sleep(5) # needed to allow the pipeline to settle in.
url = pipeline._deployment._url()
print(url)


## Test the pipeline in a video stream

### Initialize some Vars

Initialize the COCO Classes, meaning the classificaitons found on the images and the default width and height all images are resized to

In [ ]:

from CVDemoUtils import CVDemo

# set the device we will be using to run the model
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# The set of COCO classifications
CLASSES = pickle.loads(open("models/coco_classes.pickle", "rb").read())

# Unique colors for each identified COCO class
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

# The size the image will be resized to
width = 640
height = 480

# Only objects that have a confidence > confidence_target will be displayed on the image
confidence_target = 0.75

cvDemo = CVDemo(CLASSES,COLORS, DEVICE)


### Detect and Classify Anomalies in the Video Stream

Next we will load each frame in the input-video feed itto the pipeline for inferencing.  The pipeline will perform normal inferencing logic and then before returning the results execute our custom anomoly detection.  The anomalies detected, meaning their bounding boxes, their classification and confidence of classifications are included in the inferencing results.

We use these results to draw all the anomalies their bounding boxes, each identified object,  its classification, and the model's confidence that the prediction and then save the frame to an output video.


In [ ]:
# The size the image will be resized to
width = 640
height = 480

#input_video = "videos/amazon-fresh-go.mp4"
#output_video = "videos/amazon-fresh-go-inferenced.mp4"

input_video = "videos/ww2-warbirds-in-formation.mp4"
output_video = "videos/ww2-warbirds-in-formation-anomalies.mp4"

config = {
    'input-video' : input_video, # source video
    'output-video' : output_video, # show the input video with the inferenced results drawn on each frame
    'fps' : 15, # Frames per second
    'width' : width, # the width of the url
    'height' : height, # the height of the url
    #'max-frames' : 400, # the # of frames to capture in the output video
    'skip-frames' : 50, # the # of frames to capture in the output video
    'color':CVDemo.ORANGE, # color to draw bounding boxes and the text in the statistics
     #'inference' : 'WALLAROO_SDK' is only supported for anomalies right now
    'model_name' : model_name,
    'pipeline' : pipeline, # provide this when using inference WALLAROO_SDK 
    'pipeline_name' : pipeline_name,
    'record-start-frame' : 51, # the # of frames to capture in the output video
    'record-end-frame' : 151, # the # of frames to capture in the output video  
}
cvDemo.DEBUG = False
cvDemo.detectAndClassifyAnomaliesInVideo(config)
print("We are done.")




    

### Conclusion

Notice how simple it is to take the mobilenet object detectors rest api endpoint url and use it in a video stream.  Now its your turn.  Upload a video to this notebook and replace the input-video with the path of the uploaded video.  Update the output video accordingly.

See how well it works for you.  Good luck!


In [ ]:
#pipeline.undeploy()
#for d in wl.list_deployments():
#    d.undeploy()
    

Waiting for undeployment - this will take up to 45s .................................... ok
 ok
 ok
 ok
 ok
 ok
 ok
 ok
 ok
 ok
 ok
 ok
 ok
 ok
 ok
Waiting for undeployment - this will take up to 45s .............................